In [3]:
%pip install -e .


Obtaining file:///Users/szepi1991/Code/adaptive_time/code
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for adaptive_time (pyproject.toml) ... done
  Created wheel for adaptive_time: filename=adaptive_time-0.1-0.editable-py3-none-any.whl size=2569 sha256=a8be201b25daf266a493056f8b758e59a0aaa9affa47e245fd7ccafdfac5135f
  Stored in directory: /private/var/folders/xt/69nn93l959l31g_3b686nmsr0000gn/T/pip-ephem-wheel-cache-gnxt1gnm/wheels/07/43/56/a4aec053025de0632e0ed94a57fc42e37d9d73a60353c313a2
Successfully built adaptive_time


In [6]:
from adaptive_time.environment import MountainCar


In [1]:
import os
import sys

# Print the working directory
print("Working Directory:", os.getcwd())

# Print the paths for imports
print("Import Paths:")
for path in sys.path:
    print(path)


Working Directory: /Users/szepi1991/Code/adaptive_time/code
Import Paths:
/Users/szepi1991/Code/adaptive_time/code
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload

/Users/szepi1991/Code/adaptive_time/.venv/lib/python3.11/site-packages


In [2]:
print('tes')

tes
